<a href="https://colab.research.google.com/github/rodraxphysics/Master_VIU_Data_Science/blob/main/05_Caracteristicas_Derivadas_Dataset_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## <center>  <center>
## <center> Universidad Internacional de Valencia (VIU) <center>
## <center> Trabajo Fin de Master (TFM) <center>


---


# <center> **Predicción de la magnitud de la banda prohibida (bandgap) en compuestos inorgánicos mediante técnicas de Machine Learning** <center>


---



**Titulación:** Máster en Big Data y Data Science

**Alumno:** Rodrigo Eduardo Sandoval Brito

**Director:** Jose Carlos González, PhD.



---



## <center> **Generacion de caracteristicas derivadas y obtencion del dataset final** <center>

**Importacion del dataset descargado de API de Materials Project guardado en archivo csv**

In [ ]:
# Imports generales
import pandas as pd
import io
from google.colab import files
import numpy as np
import time

def upload_files (index_fields=None):
  uploaded = files.upload()
  for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))
    df = pd.read_csv(io.StringIO(uploaded[fn].decode('utf-8')), index_col = index_fields)

    return df


**Debido al exhaustivo y arduo proceso de importacion de los datos desde la API de Materials Project fue necesario descargarlos en distintos lotes, generando varios archivos csv, que los concatenaremos al final en un solo dataframe para obtener nuestro dataset final**

In [ ]:
import pandas as pd
import os

# Ruta donde se encuentran los archivos CSV
ruta_archivos = "/content/"

# Lista para almacenar los DataFrames de cada archivo CSV
dfs = []

# Recorrer la ruta proporcionada
for file_name in os.listdir(ruta_archivos):
    if file_name.endswith(".csv"):  # Verificar si es un archivo CSV
        file_path = os.path.join(ruta_archivos, file_name)
        print(f"Leyendo archivo: {file_path}")

        # Leer el CSV y agregarlo a la lista de DataFrames
        df = pd.read_csv(file_path)
        dfs.append(df)

# Concatenar todos los DataFrames en uno solo
df = pd.concat(dfs, ignore_index=True)

# Verificar las primeras filas del DataFrame combinado
print(df.head())



Leyendo archivo: /content/expanded_2208_1180995-1184328.csv
Leyendo archivo: /content/expanded_658_675010-677103.csv
Leyendo archivo: /content/expanded_1954_1187663-1190997.csv
Leyendo archivo: /content/expanded_3133_1184329-1187662.csv
Leyendo archivo: /content/expanded_189_1075010-1075954.csv
Leyendo archivo: /content/expanded_205_504510-504994.csv
Leyendo archivo: /content/expanded_1008_559871-561710.csv
Leyendo archivo: /content/materials_project.csv
  material_id formula_pretty  band_gap  nsites  nelements      volume  \
0  mp-1190996    CsGd2Cu3Se5    0.8937      22          4  539.569953   
1  mp-1180995     InCu3HgSe4    0.5689       9          4  220.636720   
2  mp-1180996        Mg2PHO5    2.2900      54          4  540.237643   
3  mp-1181000      NiP4N2O19    0.4543      52          4  874.056486   
4  mp-1181002         LaFeO3    1.2424      20          3  302.648989   

    density  density_atomic crystal_symmetry  symmetry_number  ...  \
0  6.357199       24.525907     

**Al dataset final se realizara la misma limpieza realizada anteriormente en el proceso de Extraccion de datos**

In [ ]:
df.drop('equilibrium_reaction_energy_per_atom', axis=1, inplace=True)
df=df.dropna()
df = df.drop_duplicates()
print(len(df))

29222


**Eliminamos la variable string del indice del material y la variable del numero atomico que corresponde al peso molecular, la cual sera añadida mas adelante junto con las demas caaracteristicas derivadas de la formula quimica**

In [ ]:
df.drop('material_id', axis=1, inplace=True)
df.drop('atomic_number', axis=1, inplace=True)

In [ ]:
df.head()

,material_id,formula_pretty,band_gap,nsites,nelements,volume,density,density_atomic,crystal_symmetry,symmetry_number,...,energy_per_atom,formation_energy_per_atom,energy_above_hull,is_stable,equilibrium_reaction_energy_per_atom,efermi,is_magnetic,total_magnetization,num_magnetic_sites,atomic_number
0,mp-504994,K3Cr(HO)6,3.2374,32,4,387.648980,2.324590,12.114031,Trigonal,167,...,-5.566374,-1.620641,0.005226,False,NaN,-0.432875,True,0.000000,2.0,103
1,mp-504509,VFeO4,2.0493,36,3,466.673564,3.646150,12.963155,Triclinic,2,...,-8.289074,-2.061911,0.018901,False,NaN,0.809739,True,30.000001,12.0,73
2,mp-504510,MgV2O6,2.7654,9,3,117.027435,3.152644,13.003048,Monoclinic,12,...,-8.025152,-2.523834,0.000000,True,-0.001718,2.940612,False,0.000032,0.0,75
3,mp-504520,RbEu3F10,0.0000,28,3,434.433258,5.590847,15.515474,Cubic,225,...,-7.362755,-3.722780,0.000000,True,-0.051041,-1.802077,True,36.007869,6.0,163
4,mp-504524,CsMnF4,0.0994,24,3,393.518564,4.453277,16.396607,Tetragonal,129,...,-5.885532,-2.934422,0.012583,False,NaN,0.066776,True,16.000000,4.0,107


**Realizamos el mismo proceso de conversion de variables string a numericas**

In [ ]:
mapeo = {
    'Cubic': 1,        # Mayor simetría, más simple
    'Tetragonal': 2,   # Simetría alta, ligeramente más compleja que la cúbica
    'Hexagonal': 3,    # Simetría alta, comparable a la tetragonal pero con un sistema de ejes diferente
    'Trigonal': 4,     # Similar a la hexagonal en términos de simetría, pero con diferencias en la disposición atómica
    'Orthorhombic': 5, # Menor simetría que las anteriores, con lados de diferentes longitudes y ángulos rectos
    'Monoclinic': 6,   # Aún menor simetría, con ángulos no ortogonales
    'Triclinic': 7     # La menor simetría, más compleja en términos de simetría y disposición atómica
}


# Aplica el mapeo a la columna
df['crystal_symmetry'] = df['crystal_symmetry'].replace(mapeo)

**Importamos la biblioteca de periodictable para poder generar facilmente las caracteristicas derivadas desde la formula quimica**

In [ ]:
pip install periodictable

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 752.5/752.5 kB 7.9 MB/s eta 0:00:00


De esta manera con la ayuda de la biblioteca de python periodictable y las expresiones regulares re se identificaron todos los elementos únicos de todos los compuestos del dataset y se crearon dos nuevas columnas para cada uno de ellos:


*   una columna llamada "nombre del elemento_count" que indica la cantidad de átomos de ese elemento en cada compuesto
*   otra columna llamada "nombre del elemento_mass_ratio" que representa la proporción de masa de ese elemento en cada compuesto, calculada como la masa atómica del elemento por la cantidad de átomos divido para el peso molecular del compuesto.


In [ ]:
import pandas as pd
import periodictable
import re

# Funciones previamente definidas para extraer elementos y calcular el peso molecular
def extract_elements(formula):
    pattern = r'([A-Z][a-z]*)(\d*)'
    matches = re.findall(pattern, formula)
    elements = {match[0]: int(match[1]) if match[1] else 1 for match in matches}
    return elements

def calculate_molecular_weight(elements):
    weight = 0
    for element, count in elements.items():
        weight += getattr(periodictable, element).mass * count
    return weight

# Añadiendo columnas al DataFrame
df['elements'] = df['formula_pretty'].apply(extract_elements)
df['molecular_weight'] = df['elements'].apply(calculate_molecular_weight)

# Expandiendo el DataFrame para incluir la proporción de masa y la cantidad de átomos
for element in set().union(*df['elements']):
    df[f'{element}_count'] = df['elements'].apply(lambda x: x.get(element, 0))
    df[f'{element}_mass_ratio'] = df.apply(lambda row: (getattr(periodictable, element).mass * row[f'{element}_count']) / row['molecular_weight'] if row[f'{element}_count'] > 0 else 0, axis=1)

# Limpiando el DataFrame para el ejemplo
df.drop(columns=['elements'], inplace=True)

df.head()

,material_id,formula_pretty,band_gap,nsites,nelements,volume,density,density_atomic,crystal_symmetry,symmetry_number,...,He_count,He_mass_ratio,Zr_count,Zr_mass_ratio,Np_count,Np_mass_ratio,Ta_count,Ta_mass_ratio,Pb_count,Pb_mass_ratio
0,mp-561710,CsSnS3,1.6557,10,3,322.270798,3.584270,32.227080,7,2,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
1,mp-559871,AlF3,7.5231,24,2,290.558390,2.879559,12.106600,5,63,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
2,mp-559872,SiO2,5.6106,36,2,491.816081,2.434381,13.661558,2,96,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
3,mp-559873,CSNO2F3,4.7073,96,5,1498.617602,1.955611,15.610600,6,14,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
4,mp-559874,FeTe2ClO5,0.0230,72,4,1224.710534,4.626149,17.009869,6,14,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0


**Eliminamos las variables que ya no contribuyen con valores numericos y transformamos las variables booleanas a enteras**

In [ ]:
df.drop('material_id', axis=1, inplace=True)
df.drop('formula_pretty', axis=1, inplace=True)

In [ ]:
df['is_magnetic'] = df['is_magnetic'].astype(int)
df['is_stable'] = df['is_stable'].astype(int)

**De esta manera obtenemos un dataset final con cerca de 30 mil instancias y cerca de 200 caracteristicas**

In [ ]:
print(df.dtypes)
print(df.shape)

band_gap         float64
nsites             int64
nelements          int64
volume           float64
density          float64
                  ...   
Np_mass_ratio    float64
Ta_count           int64
Ta_mass_ratio    float64
Pb_count           int64
Pb_mass_ratio    float64
Length: 199, dtype: object
(29229, 199)


**Comprobamos tambien que el dataset se encuentra limpio y no contiene ningun dato nulo**

In [ ]:
nulos_por_columna = df.isnull().sum()
print(nulos_por_columna)

band_gap         0
nsites           0
nelements        0
volume           0
density          0
                ..
Np_mass_ratio    0
Ta_count         0
Ta_mass_ratio    0
Pb_count         0
Pb_mass_ratio    0
Length: 199, dtype: int64


**Exportamos el dataset final en un archivo csv**

In [ ]:
df.to_csv('dataframe_TRATADO_completo.csv', index=False)